In [1]:
# install spark
!pip install pyspark

In [2]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/05 17:17:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Load the flight dataset
src = '/Users/fengzheng/Desktop/Coding/Practice/CS5540Project/data_raw/flight_raw'
flight_df = spark.read.option('header','true').csv(src)

# Create timestamp column
from pyspark.sql.functions import *
flight_df = flight_df.withColumn('Date',to_date(concat_ws('-', flight_df.Year, flight_df.Month,flight_df.DayofMonth)))

# Select needed columns
flight_df = flight_df.select('Date','Airline','Origin','Dest','Cancelled','OriginStateName','DestStateName')\
                        .sort('Date')

# Print schema
flight_df.printSchema()
flight_df.show(5)

root
 |-- Date: date (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- DestStateName: string (nullable = true)



+----------+--------------------+------+----+---------+---------------+-------------+
|      Date|             Airline|Origin|Dest|Cancelled|OriginStateName|DestStateName|
+----------+--------------------+------+----+---------+---------------+-------------+
|2019-01-01|ExpressJet Airlin...|   IAH| BTR|    False|          Texas|    Louisiana|
|2019-01-01|ExpressJet Airlin...|   ORD| SDF|    False|       Illinois|     Kentucky|
|2019-01-01|ExpressJet Airlin...|   LFT| IAH|    False|      Louisiana|        Texas|
|2019-01-01|ExpressJet Airlin...|   IAH| VPS|     True|          Texas|      Florida|
|2019-01-01|ExpressJet Airlin...|   GPT| IAH|    False|    Mississippi|        Texas|
+----------+--------------------+------+----+---------+---------------+-------------+
only showing top 5 rows



In [4]:
# Load the covid dataset
src2 = '/Users/fengzheng/Desktop/Coding/Practice/CS5540Project/data_raw/covid_raw/'
covid_df = spark.read.option('header','true').csv(src2)

# Create timestamp column
from pyspark.sql.functions import *
covid_df = covid_df.withColumn('Date',to_date(covid_df.date))

# Select needed columns
covid_df = covid_df.select('Date','state','cases','deaths')\
                        .sort('Date')

# Print schema
covid_df.printSchema()
covid_df.show(5)

root
 |-- Date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- cases: string (nullable = true)
 |-- deaths: string (nullable = true)

+----------+----------+-----+------+
|      Date|     state|cases|deaths|
+----------+----------+-----+------+
|2020-01-21|Washington|    1|     0|
|2020-01-22|Washington|    1|     0|
|2020-01-23|Washington|    1|     0|
|2020-01-24|Washington|    1|     0|
|2020-01-24|  Illinois|    1|     0|
+----------+----------+-----+------+
only showing top 5 rows



In [5]:
# submit the cleaned dataset
covid_df.repartition(1).write.format("com.databricks.spark.csv").option("header","true").save("/Users/fengzheng/Desktop/covid_clean")

In [6]:
# submit the cleaned dataset
flight_df.repartition(1).write.format("com.databricks.spark.csv").option("header","true").save("/Users/fengzheng/Desktop/flight_clean")